In [15]:
import os
import cv2

face_cascade = cv2.CascadeClassifier(""" haar cascade 모델 위치 """)
blue_color = (255, 0, 0)

## face box 

In [16]:
def detect_face(img) :
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    return faces

def draw_face_rec(img, faces) :
    if len(faces) == 0:
        print("No Face Detected")
    else :
        for (x, y, w, h) in faces :
            cv2.rectangle(img, (x, y), (x+w, y+h), blue_color, 2)
    
    return img

def show_image(title, img) :
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [4]:
img_file = """ 이미지 path """
img = cv2.imread(img_file)

faces = detect_face(img)
result = draw_face_rec(img, faces)
show_image(""" window title """, result)

## face noonsong

In [17]:
def make_border(img) :
    h, w, c = img.shape
    border = int(max(h, w) / 2)
    img = cv2.copyMakeBorder(img, border, border, border, border, cv2.BORDER_CONSTANT)
    return img, border

def delete_border(img, border) :
    h, w, c = img.shape
    cropped_img = img[border:h-border, border:w-border]
    return cropped_img

def make_loc(face, noonsong) :
    x, y, w, h = face
    origin_h, origin_w, origin_c = noonsong.shape
    ns_w = int(w*2.0)
    ns_h = int(origin_h * ns_w / origin_w)
    noonsong = cv2.resize(noonsong, dsize=(ns_w, ns_h), interpolation = cv2.INTER_AREA)
    
    loc_x = x - int(ns_w * 0.2)
    loc_y = y + h - ns_h
    if loc_y < 0 : loc_y = 0
        
    return loc_x, loc_y, ns_w, ns_h, noonsong
    
    
def draw_noonsong(img, faces) :
    noonsong = cv2.imread(""" overlay 이미지 path""")
    
    for face in faces :
        x, y, w, h, noonsong = make_loc(face, noonsong)
        
        noonsong_gray = cv2.cvtColor(noonsong, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(noonsong_gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        
        roi = img[y:y+h, x:x+w]
        img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        noonsong_bg = cv2.bitwise_and(noonsong, noonsong, mask=mask)
        
        dst = cv2.add(img_bg, noonsong_bg)
        img[y:y+h, x:x+w] = dst
    
    return img

In [10]:
img_file = """ 이미지 path """
img = cv2.imread(img_file)

img, border = make_border(img)
faces = detect_face(img)
result = draw_noonsong(img, faces)
result = delete_border(result, border)
show_image(""" window title """, result)

## face box _ web cam

In [23]:
def webcam_init() :
    cap = cv2.VideoCapture(0)
    cap.set(3, 720)
    cap.set(4, 1080)
    return cap

def draw_face_rec(img, faces) :
    for (x, y, w, h) in faces :
        cv2.rectangle(img, (x, y), (x+w, y+h), blue_color, 2)
    
    return img

In [20]:
cap = webcam_init()

while True :
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    faces = detect_face(frame)
    frame = draw_face_rec(frame, faces)

    cv2.imshow("""window title""", frame)
    key = cv2.waitKey(1)
    if key & 0xFF == 27 :
        break
        
cap.release()
cv2.destroyAllWindows()

## face noonsong _ web cam

In [25]:
def draw_noonsong(img, noonsong, faces) :
    
    for face in faces :
        x, y, w, h, noonsong = make_loc(face, noonsong)
        
        noonsong_gray = cv2.cvtColor(noonsong, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(noonsong_gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        
        roi = img[y:y+h, x:x+w]
        img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        noonsong_bg = cv2.bitwise_and(noonsong, noonsong, mask=mask)
        
        dst = cv2.add(img_bg, noonsong_bg)
        img[y:y+h, x:x+w] = dst
    
    return img

In [26]:
cap = webcam_init()
noonsong = cv2.imread(""" overlay 이미지 path""")

while True :
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    frame, border = make_border(frame)
    faces = detect_face(frame)
    if len(faces) != 0 :
        frame = draw_noonsong(frame, noonsong, faces)
    frame = delete_border(frame, border)
    
    cv2.imshow(""" window title """, frame)
    
    key = cv2.waitKey(1)
    if key & 0xFF == 27 :
        break
        
cap.release()
cv2.destroyAllWindows()